## Loading Libraries
All Python capabilities are not loaded to our working environment by default (even they are already installed in your system). So, we import each and every library that we want to use.

We chose alias names for our libraries for the sake of our convenience (numpy --> np and pandas --> pd, tensorlow --> tf).

Note: You can import all the libraries that you think will be required or can import it as you go along.

In [1]:
import pandas as pd                                     # Data analysis and manipultion tool
import numpy as np                                      # Fundamental package for linear algebra and multidimensional arrays
import tensorflow as tf                                 # Deep Learning Tool
import os                                               # OS module in Python provides a way of using operating system dependent functionality
import cv2                                              # Library for image processing
from sklearn.model_selection import train_test_split    # For splitting the data into train and validation set

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Loading and preparing training data
The train and test images are given in two different folders - 'train' and 'test'. The labels of train images are given in a csv file 'Train.csv' with respective image id (i.e. image file name).

#### Getting the labels of the images

In [3]:
labels = pd.read_csv("/content/drive/MyDrive/Butterfly Identification/Training_set.csv")   # loading the labels
labels.head()           # will display the first five rows in labels dataframe

,filename,label
0,Image_1.jpg,SOUTHERN DOGFACE
1,Image_2.jpg,ADONIS
2,Image_3.jpg,BROWN SIPROETA
3,Image_4.jpg,MONARCH
4,Image_5.jpg,GREEN CELLED CATTLEHEART


In [4]:
labels.tail()            # will display the last five rows in labels dataframe

,filename,label
6494,Image_6495.jpg,MANGROVE SKIPPER
6495,Image_6496.jpg,MOURNING CLOAK
6496,Image_6497.jpg,APPOLLO
6497,Image_6498.jpg,ELBOWED PIERROT
6498,Image_6499.jpg,ATALA


#### Getting images file path

In [5]:
file_paths = [[fname, '/content/drive/MyDrive/Butterfly Identification/train/' + fname] for fname in labels['filename']]

#### Confirming if no. of labels is equal to no. of images

In [6]:
# Confirm if number of images is same as number of labels given
if len(labels) == len(file_paths):
    print('Number of labels i.e. ', len(labels), 'matches the number of filenames i.e. ', len(file_paths))
else:
    print('Number of labels does not match the number of filenames')

Number of labels i.e.  6499 matches the number of filenames i.e.  6499


#### Converting the file_paths to dataframe

In [7]:
images = pd.DataFrame(file_paths, columns=['filename', 'filepaths'])
images.head()

,filename,filepaths
0,Image_1.jpg,/content/drive/MyDrive/Butterfly Identificatio...
1,Image_2.jpg,/content/drive/MyDrive/Butterfly Identificatio...
2,Image_3.jpg,/content/drive/MyDrive/Butterfly Identificatio...
3,Image_4.jpg,/content/drive/MyDrive/Butterfly Identificatio...
4,Image_5.jpg,/content/drive/MyDrive/Butterfly Identificatio...


#### Combining the labels with the images

In [8]:
train_data = pd.merge(images, labels, how = 'inner', on = 'filename')
train_data.head()       

,filename,filepaths,label
0,Image_1.jpg,/content/drive/MyDrive/Butterfly Identificatio...,SOUTHERN DOGFACE
1,Image_2.jpg,/content/drive/MyDrive/Butterfly Identificatio...,ADONIS
2,Image_3.jpg,/content/drive/MyDrive/Butterfly Identificatio...,BROWN SIPROETA
3,Image_4.jpg,/content/drive/MyDrive/Butterfly Identificatio...,MONARCH
4,Image_5.jpg,/content/drive/MyDrive/Butterfly Identificatio...,GREEN CELLED CATTLEHEART


In [9]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train_data['label'] = le.fit_transform(train_data['label'])

The 'train_data' dataframe contains all the image id, their locations and their respective labels. Now the training data is ready.

## Data Pre-processing
It is necessary to bring all the images in the same shape and size, also convert them to their pixel values because all machine learning or deep learning models accepts only the numerical data. Also we need to convert all the labels from categorical to numerical values.

In [10]:
data = []     # initialize an empty numpy array
image_size = 100      # image size taken is 100 here. one can take other size too
for i in range(len(train_data)):
  
  img_array = cv2.imread(train_data['filepaths'][i], cv2.IMREAD_GRAYSCALE)   # converting the image to gray scale

  new_img_array = cv2.resize(img_array, (image_size, image_size))      # resizing the image array
  data.append([new_img_array, train_data['label'][i]])

In [11]:
4# image pixels of a image
data[5]

[array([[ 65,  67,  69, ...,  85,  87,  88],
        [ 65,  66,  67, ...,  88,  87,  88],
        [ 60,  61,  65, ...,  88,  88,  90],
        ...,
        [111, 110, 111, ..., 114, 114, 117],
        [108, 110, 112, ..., 112, 113, 118],
        [109, 108, 111, ..., 111, 113, 121]], dtype=uint8), 14]

#### Shuffle the data

In [12]:
np.random.shuffle(data)

#### Separating the images and labels


In [13]:
x = []
y = []
for image in data:
  x.append(image[0])
  y.append(image[1])

# converting x & y to numpy array as they are list
x = np.array(x)
y = np.array(y)

In [14]:
np.unique(y, return_counts=True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
        51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
        68, 69, 70, 71, 72, 73, 74]),
 array([ 88,  75,  74,  85,  90, 100,  97,  83,  81,  85,  75,  86,  99,
         90,  83,  95,  85,  93,  87,  92,  87,  90,  94,  97,  72,  94,
         93,  92,  95,  82,  73,  78,  94,  88,  86,  81,  95,  81,  81,
         73,  87,  86,  76,  96,  90, 131,  87,  96,  76,  78,  90,  84,
         86,  84,  85,  79,  92,  77,  82,  96,  89,  86,  97,  83, 107,
         90,  87,  87,  83,  76,  84,  81,  71,  75,  76]))

#### Splitting the data into Train and Validation Set
We want to check the performance of the model that we built. For this purpose, we always split (both independent and dependent data) the given data into training set which will be used to train the model, and test set which will be used to check how accurately the model is predicting outcomes.

For this purpose we have a class called 'train_test_split' in the 'sklearn.model_selection' module.

In [15]:
x =  x.reshape(-1, 100, 100, 1)

In [16]:
# split the data
X_train, X_val, y_train, y_val = train_test_split(x,y,test_size=0.3, random_state = 42)

## Building Model
Now we are finally ready, and we can train the model.

There are many machine learning or deep learning models like Random Forest, Decision Tree, Multi-Layer Perceptron (MLP), Convolution Neural Network (CNN), etc. to say you some.


Then we would feed the model both with the data (X_train) and the answers for that data (y_train)

In [20]:
cnn = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
   # tf.keras.layers.Dropout((0.25)),
    
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
   #tf.keras.layers.Dropout((0.25)),

    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    #tf.keras.layers.Dropout((0.25)),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(75, activation='softmax')
])

In [21]:
cnn.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [22]:
cnn.fit(X_train, y_train, epochs=35, batch_size=64,validation_data=(X_val, y_val))

Epoch 1/35
72/72 [==============================] - 78s 1s/step - loss: 7.7406 - accuracy: 0.0299 - val_loss: 4.1255 - val_accuracy: 0.0626
Epoch 2/35
72/72 [==============================] - 76s 1s/step - loss: 3.6804 - accuracy: 0.1444 - val_loss: 3.6513 - val_accuracy: 0.1626
Epoch 3/35
72/72 [==============================] - 76s 1s/step - loss: 2.8826 - accuracy: 0.2963 - val_loss: 3.5043 - val_accuracy: 0.2000
Epoch 4/35
72/72 [==============================] - 76s 1s/step - loss: 2.0895 - accuracy: 0.4564 - val_loss: 3.3234 - val_accuracy: 0.2610
Epoch 5/35
72/72 [==============================] - 76s 1s/step - loss: 1.4625 - accuracy: 0.6058 - val_loss: 3.5868 - val_accuracy: 0.2779
Epoch 6/35
72/72 [==============================] - 76s 1s/step - loss: 0.9727 - accuracy: 0.7311 - val_loss: 4.5085 - val_accuracy: 0.2713
Epoch 7/35
72/72 [==============================] - 76s 1s/step - loss: 0.6887 - accuracy: 0.8048 - val_loss: 4.7543 - val_accuracy: 0.2723
Epoch 8/35
72/72 [==

## Validate the model
Wonder🤔 how well your model learned! Lets check its performance on the X_val data.

In [23]:
cnn.evaluate(X_val, y_val)

61/61 [==============================] - 8s 133ms/step - loss: 11.6049 - accuracy: 0.3262


[11.60488224029541, 0.32615384459495544]

## Predict The Output For Testing Dataset 😅
We have trained our model, evaluated it and now finally we will predict the output/target for the testing data (i.e. Test.csv).

#### Load Test Set
Load the test data on which final submission is to be made.

In [24]:
# Loading the order of the image's name that has been provided
test_image_order = pd.read_csv("/content/drive/MyDrive/Butterfly Identification/Testing_set.csv")
test_image_order.head()

,filename
0,Image_1.jpg
1,Image_2.jpg
2,Image_3.jpg
3,Image_4.jpg
4,Image_5.jpg


#### Getting images file path

In [25]:
file_paths = [[fname, '/content/drive/MyDrive/Butterfly Identification/test/' + fname] for fname in test_image_order['filename']]

#### Confirm if number of images in test folder is same as number of image names in 'Testing_set_face_mask.csv'

In [26]:
# Confirm if number of images is same as number of labels given
if len(test_image_order) == len(file_paths):
    print('Number of image names i.e. ', len(test_image_order), 'matches the number of file paths i.e. ', len(file_paths))
else:
    print('Number of image names does not match the number of filepaths')

Number of image names i.e.  2786 matches the number of file paths i.e.  2786


#### Converting the file_paths to dataframe

In [27]:
test_images = pd.DataFrame(file_paths, columns=['filename', 'filepaths'])
test_images.head()

,filename,filepaths
0,Image_1.jpg,/content/drive/MyDrive/Butterfly Identificatio...
1,Image_2.jpg,/content/drive/MyDrive/Butterfly Identificatio...
2,Image_3.jpg,/content/drive/MyDrive/Butterfly Identificatio...
3,Image_4.jpg,/content/drive/MyDrive/Butterfly Identificatio...
4,Image_5.jpg,/content/drive/MyDrive/Butterfly Identificatio...


## Data Pre-processing on test_data


In [28]:
test_pixel_data = []     # initialize an empty numpy array
image_size = 100      # image size taken is 100 here. one can take other size too
for i in range(len(test_images)):
  
  img_array = cv2.imread(test_images['filepaths'][i], cv2.IMREAD_GRAYSCALE)   # converting the image to gray scale

  new_img_array = cv2.resize(img_array, (image_size, image_size))      # resizing the image array

  test_pixel_data.append(new_img_array)

In [29]:
test_pixel_data = np.array(test_pixel_data)

In [30]:
test_pixel_data =  test_pixel_data.reshape(-1, 100, 100, 1)

### Make Prediction on Test Dataset
Time to make a submission!!!

In [31]:
pred = cnn.predict(test_pixel_data)

In [32]:
# The predicted values are the probabilities value
pred[0]

array([4.70117482e-14, 7.48873664e-29, 9.89302613e-16, 2.13228320e-22,
       2.06428282e-16, 9.60698178e-33, 5.71566242e-19, 3.67297415e-10,
       4.88598880e-06, 2.64119475e-31, 3.18109139e-09, 5.17654901e-19,
       3.22223374e-19, 9.61310430e-18, 2.17840306e-14, 1.88786990e-11,
       8.87965235e-18, 1.53085573e-20, 1.01170547e-22, 7.35140109e-22,
       9.28306668e-11, 2.02651944e-16, 1.41023905e-24, 1.76804003e-24,
       2.24527576e-16, 6.63678973e-11, 3.98943571e-16, 9.29508274e-14,
       1.06562942e-32, 2.46714658e-17, 7.12330611e-11, 3.90673797e-17,
       3.31633673e-26, 1.34752591e-14, 2.99235753e-20, 1.16144761e-17,
       1.63456807e-05, 1.40471997e-16, 6.04096580e-14, 7.98109951e-08,
       2.79561424e-15, 1.96276084e-02, 4.59234074e-11, 5.78207355e-24,
       1.17524513e-26, 1.40497257e-30, 1.43051219e-19, 1.03372770e-07,
       2.25425956e-10, 1.55263599e-32, 2.78048694e-25, 4.36483543e-25,
       2.51100188e-18, 1.27459350e-25, 3.19287282e-31, 5.52622294e-17,
      

The above values are probability values. We need to convert it into respective classes. We can use np.argmax for the same.

In [33]:
prediction = []
for value in pred:
  prediction.append(np.argmax(value))

In [34]:
predictions = le.inverse_transform(prediction)

In [35]:
res = pd.DataFrame({'filename': test_images['filename'], 'label': predictions})  # prediction is nothing but the final predictions of your model on input features of your new unseen test data
res.to_csv("submission.csv", index = False)      # the csv file will be saved locally on the same location where this notebook is located.